# Laboratorio 2.2: Clasificación

Bárbara Poblete, Felipe Bravo, Aymé Arango, Juglar Díaz, Hernán Sarmiento, Juan Pablo Silva
**Septiembre 2019**

## =================== INTEGRANTES =====================

Escriba a continuación el nombre de los integrantes del presente laboratorio:

1. Matías Rojas

2. David de la Puente

## =====================================================

# Instrucciones


1. El formato de entrega es un documento en **.html**, generado por jupyter.

2. El laboratorio debe realizarse en grupos de **2 personas**.

3. Asegúrese que están los nombres de los integrantes. Sólo uno de los integrantes debe subir este archivo a U-Cursos antes de finalizar la sesión. 

4. Las respuestas a cada pregunta se deben escribir en los bloques que dicen **RESPUESTA A PREGUNTA X.X**.

# Del Laboratorio 

En este laboratorio vamos a comparar clasificadores con cierto *baselines* o clasificadores base, y además vamos a trabajar con clases desbalanceadas. 

# Parte 1: Comparar clasificadores

Una de las principales tareas en enfoques supervisados es evaluar diferentes clasificadores y encontrar el mejor de alguno de ellos para un problema. Por ejemplo, si tenemos dos (o más) clasificadores y queremos compararlos entre sí, nos interesa responder: *¿Cuál de los clasificadores es el mejor?* 
Para responder esta pregunta, no existe una única solución. 

Lo que haremos a continuación será ejecutar diferentes clasificadores y compararlos en base a las métricas de Precision, Recall y F1-score.

## Pregunta 1.1  

Para realizar la evaluación de distintos clasificadores, vamos a crear la función `run_classifier()`, la cual evalúa un clasificador `clf` recibido como parámetro un dataset `X,y` (dividido en training y testing) y un número de tests llamado `num_test`. Esta función almacena y retorna los valores de precision, recall y f1-score en la variable `metrics` además de los resultados de predicción.


En base a lo anterior, incluya las sentencias que ajusten el modelo junto a su correspondiente predicción sobre los datos. No use cross-validation ni tampoco el parámetro `random_state`.


### Respuesta 1.1

In [0]:
### COMPLETAR ESTE CÓDIGO

## run_classifier recibe un clasificador y un dataset dividido para entrenamiento y testing
## y opcionalmente la cantidad de resultados que se quiere obtener del clasificador

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score


def run_classifier(clf, X, y, num_tests=100):
    metrics = {'f1-score': [], 'precision': [], 'recall': []}
    

    
    for i in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)
        ### INICIO COMPLETAR ACÁ 
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)   
        metrics['y_pred'] = predictions
        metrics['y_prob'] = clf.predict_proba(X_test)[:,1]
        metrics['f1-score'].append(f1_score(y_test, predictions)) 
        metrics['recall'].append(recall_score(y_test, predictions))
        metrics['precision'].append(precision_score(y_test, predictions))
    
    return metrics

Luego de completar el código anterior, ejecute el siguiente bloque para comparar los distintos clasificadores. 
Usaremos un **dataset de cáncer de mamas** para evaluar. Información del dataset la puede encontrar en el siguiente link: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [0]:
## ejecutar este código

from sklearn.datasets import load_breast_cancer
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier

bc = load_breast_cancer()    # dataset cancer de mamas
X = bc.data
y = bc.target

c0 = ("Base Dummy", DummyClassifier(strategy='stratified'))
c1 = ("Decision Tree", DecisionTreeClassifier())
c2 = ("Gaussian Naive Bayes", GaussianNB())
c3 = ("KNN", KNeighborsClassifier(n_neighbors=5))

classifiers = [c0,c1, c2, c3]

results = {}
for name, clf in classifiers:
    metrics = run_classifier(clf, X, y)   # hay que implementarla en el bloque anterior.
    results[name] = metrics
    print("----------------")
    print("Resultados para clasificador: ",name) 
    print("Precision promedio:",np.array(metrics['precision']).mean())
    print("Recall promedio:",np.array(metrics['recall']).mean())
    print("F1-score promedio:",np.array(metrics['f1-score']).mean())
    print("----------------\n\n")
    

----------------
Resultados para clasificador:  Base Dummy
Precision promedio: 0.6345476436166589
Recall promedio: 0.6219521025007758
F1-score promedio: 0.6268889039891483
----------------


----------------
Resultados para clasificador:  Decision Tree
Precision promedio: 0.9445022729992467
Recall promedio: 0.940597872560335
F1-score promedio: 0.9421697960938029
----------------


----------------
Resultados para clasificador:  Gaussian Naive Bayes
Precision promedio: 0.9396517716275639
Recall promedio: 0.969252152439604
F1-score promedio: 0.9540080318144442
----------------


----------------
Resultados para clasificador:  KNN
Precision promedio: 0.9314819766155358
Recall promedio: 0.961666151976116
F1-score promedio: 0.9460688309286859
----------------




### Pregunta 1.2

Analizando los resultados obtenidos de cada clasificador, y basándose en las métricas calculadas. ¿Cuál es el mejor clasificador? ¿Qué métricas observó para tomar esa decisión y por qué? Fundamente su respuesta.

### Respuesta 1.2

En base a lo observado, podemos concluir que el mejor clasificador para este caso es el Gaussian Naive Bayes. Esto porque al ver las tres métricas utilizadas, tiene mejores resultados en comparación al resto tanto en Recall como en F1-score, y como esto está hecho en base a un promedio de varias iteraciones significa que en general se comportará así de bien. Si bien no alcanza el mayor puntaje en cuanto a precisión, está muy cercano al mayor que es el de árbol de decisión.


#Parte 2: Seleccionando hiperparámetros
Los hiperparámetros son parámetros que no se aprenden directamente dentro de los estimadores. En scikit-learn se pasan como argumentos al constructor de las clases. Por ejemplo que kernel usar para Support Vector Classifier, o que criterion para Decision Tree, etc. Es posible y recomendable buscar en el espacio de hiperparámetros la mejor alternativa. Cualquier parámetro proporcionado al construir un estimador puede optimizarse de esta manera. Para encontrar los nombres y los valores actuales de todos los parámetros para un estimador dado puede usar *estimator.get_params()*.

Una búsqueda consiste en:

*   un estimador (regresor o clasificador como sklearn.svm.SVC ());
*   un espacio de parámetros;
*   un método para buscar o muestrear candidatos;
*   un esquema de validación cruzada; y
*   una función de puntuación(score).


Tenga en cuenta que es común que un pequeño subconjunto de esos parámetros pueda tener un gran impacto en el rendimiento predictivo o de cálculo del modelo, mientras que otros pueden dejar sus valores predeterminados. Se recomienda leer la documentación de la clase de estimador para obtener una mejor comprensión de su comportamiento esperado, posiblemente leyendo la referencia adjunta a la literatura.

###Pregunta 2.1 

Una alternativa para seleccionar hiperparámetros es GridSearchCV. GridSearchCV considera exhaustivamente todas las combinaciones de parámetros. GridSearchCV recibe un *estimador*, recibe *param_grid* (un diccionario o una lista de diccionarios con los nombres de los parametros a probar como keys y una lista de los valores a probar), *scoring* una o varias funciones de puntuación (score) para evaluar cada combinación de parametros y *cv* una extrategia para hacer validación cruzada.

El siguiente código muestra como seleccionar el número de vecinos y que pesos otorgar a los vecinos en un clasificador KNN. 
 


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

tuned_parameters = {'n_neighbors': [1,3,5], 'weights': ['uniform','distance']}
score = 'precision'

clf = GridSearchCV(KNeighborsClassifier(), param_grid=tuned_parameters, cv=5,
                       scoring=score)
clf.fit(X_train, y_train)

print("Mejor combinación de parámetros:")
print(clf.best_params_)
 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

Mejor combinación de parámetros:
{'n_neighbors': 3, 'weights': 'uniform'}
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        58
           1       0.94      0.96      0.95       113

    accuracy                           0.94       171
   macro avg       0.93      0.92      0.93       171
weighted avg       0.94      0.94      0.94       171



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


###Pregunta
*  a) Realice este mismo proceso para un clasificador DecisionTree y los parametros criterion=['gini','entropy'] y max_depth=[1,3,5].
*  b) ¿Qué puede decir de los resultados, considera que es necesario seguir explorando los parámetros, fue útil hacer este análisis?

In [0]:



## RESPUESTA A PREGUNTA 2.1 a)

#Completar codigo aca
tuned_parameters = {'max_depth': [1,3,5],  'criterion': ['gini','entropy']} #Completar tuned_parameters
score = 'precision'
clf = GridSearchCV(DecisionTreeClassifier(), param_grid=tuned_parameters, cv=5,
                       scoring=score)
clf.fit(X_train, y_train)


print("Mejor combinación de parámetros:")
print(clf.best_params_)
 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

Mejor combinación de parámetros:
{'criterion': 'gini', 'max_depth': 5}
              precision    recall  f1-score   support

           0       0.94      0.88      0.91        58
           1       0.94      0.97      0.96       113

    accuracy                           0.94       171
   macro avg       0.94      0.93      0.93       171
weighted avg       0.94      0.94      0.94       171



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Respuesta 2.1 b)

Se ejecutó varias veces este trozo de código para ver que configuraciones arrojaba. Pudimos apreciar que la mejor configuración en la mayoría de las ocaciones es con criterion='entropy' y max_depth=5, pero aún así al ejecutarlo varias veces entrega en ciertas ocasiones la configuración criterion='gini' y max_depth=5, pero pocas veces en comparación a la otra, es por esto que sería bueno seguir explorando los hiperparámetros para ver como influyen en las métricas. A simple vista, uno de los factores más importantes es la máxima profundidad del árbol, probamos con nuevos valores en el max_depth (3,5,10) y se escogió como mejor configuración los modelos con profundidad mayor: 10.


---

# Parte 3: Tratando con clases desbalanceadas

Para mejorar el rendimiento de un clasificador sobre clases desbalanceadas existen varias técnicas. En esta parte, veremos cómo tratar con este problema usando (sub/over)sampling de las clases.

Descargue el dataset `unbalanced.csv` que está en el tutorial. 

(*Nota: Para ejecutar el siguiente bloque es necesaria la librería `pandas` que viene incluida en Anaconda.*)

In [0]:
import pandas as pd

# Cargamos dataset desbalanceado
unbalanced = 'unbalanced.csv'
unbalanced = 'https://users.dcc.uchile.cl/~hsarmien/mineria/datasets/unbalanced.csv'

data = pd.read_csv(unbalanced)  # abrimos el archivo csv y lo cargamos en data.
data.head()

,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,Class
0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,0
1,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,1
2,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,0
3,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,1
4,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,0


Note el desbalance de las clases ejecutando el siguiente código:

In [0]:
print("Distribucion de clases original")
data['Class'].value_counts()

Distribucion de clases original


0    225
1    126
Name: Class, dtype: int64

Antes de hacer algo para tratar el desbalance entre las clases debemos antes dividir en train-test.

In [0]:
data_train, data_test, ytrain, ytest = train_test_split(data, data['Class'], test_size=0.2, stratify=data['Class'])

Así queda la proporción de clases en el train después de dividir en train-test.

In [0]:
ytrain.value_counts()

0    179
1    101
Name: Class, dtype: int64

Ahora, usando el dataset anterior, aplicaremos **oversampling** y **subsampling** al train para que queden balanceados. Ejecute el siguiente código y note ahora que las clases están balanceadas. 

In [0]:
import numpy as np

print("Distribución de clases usando (over/sub)sampling")
print()

data_train = data_train.reset_index(drop=True)

# oversampling sobre la clase 1
idx = np.random.choice(data_train[data_train['Class'] == 1].index, size=78)
data_oversampled = pd.concat([data_train, data_train.iloc[idx]])
print("Data oversampled on class '1'")
print(data_oversampled['Class'].value_counts())
print()


# subsampling sobre la clase 0
idx = np.random.choice(data_train.loc[data_train.Class == 0].index, size=78, replace=False)
data_subsampled = data_train.drop(data_train.iloc[idx].index)
print("Data subsampled on class '0'")
print(data_subsampled['Class'].value_counts())

Distribución de clases usando (over/sub)sampling

Data oversampled on class '1'
1    179
0    179
Name: Class, dtype: int64

Data subsampled on class '0'
1    101
0    101
Name: Class, dtype: int64


Para la siguiente pregunta, vamos a entrenar un árbol de decisión (`DecisionTreeClassifier`) sobre los 3 datasets por separado (**original**, con **oversampling** y con **subsampling**) y luego comparamos los resultados usando alguna métrica de evaluación.

Ejecute el siguiente bloque para cargar los datos:

In [0]:
## ejecutar este código para preparar los datos
from sklearn.metrics import classification_report

# Preparando los data frames para ser compatibles con sklearn

# datos test
X_test = data_test[data_train.columns[:-1]] # todo hasta la penultima columna
y_test = data_test[data_train.columns[-1]]  # la última columna


# datos entrenamiento "originales"
X_orig = data_train[data_train.columns[:-1]] 
y_orig = data_train[data_train.columns[-1]] 

# datos entrenamiento "oversampleados" 
X_over = data_oversampled[data_train.columns[:-1]]
y_over = data_oversampled[data_train.columns[-1]]

# datos entrenamiento "subsampleados"
X_subs = data_subsampled[data_train.columns[:-1]]
y_subs = data_subsampled[data_train.columns[-1]]

## Pregunta 3.1

Complete el código necesario para ejecutar el clasificador en cada uno de los tres casos. Emplee como datos de entrada lo del bloque anterior. Para cada caso entrene con el dataset correspondiente y evalue con el conjunto de test (será el mismo para los tres casos) obtenido con train_test_split sobre los datos originales. 

Muestre Precision, Recall y F1-score.


### RESPUESTA PREGUNTA 3.1 (agregue código en el siguiente bloque)

In [0]:
## RESPUESTA A PREGUNTA 3.1

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Recuerde:
##  - instanciar el clasificador con DecisionTreeClassifier()
##  - entrenar con fit()
##  - hacer las predicciones
##  - Mostrar precision, recall y f1-score.


# Aca esta el codigo usando el dataset: original 
print("ORIGINAL::::::::::")
clf_orig = DecisionTreeClassifier()

clf_orig.fit(X_orig,y_orig)
pred_orig = clf_orig.predict(X_test)
print(classification_report(y_test, pred_orig))

# Complete el resto para oversampling y subsampling 


print("OVERSAMPLING::::::::::")

clf_over = DecisionTreeClassifier()

clf_over.fit(X_over,y_over)
pred_over = clf_over.predict(X_test)
print(classification_report(y_test, pred_over))

print("SUBSAMPLING::::::::::")


clf_subs = DecisionTreeClassifier()

clf_subs.fit(X_subs,y_subs)
pred_subs = clf_subs.predict(X_test)
print(classification_report(y_test, pred_subs))



ORIGINAL::::::::::
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        46
           1       0.85      0.88      0.86        25

    accuracy                           0.90        71
   macro avg       0.89      0.90      0.89        71
weighted avg       0.90      0.90      0.90        71

OVERSAMPLING::::::::::
              precision    recall  f1-score   support

           0       0.86      0.91      0.88        46
           1       0.82      0.72      0.77        25

    accuracy                           0.85        71
   macro avg       0.84      0.82      0.83        71
weighted avg       0.84      0.85      0.84        71

SUBSAMPLING::::::::::
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        46
           1       0.72      0.84      0.78        25

    accuracy                           0.83        71
   macro avg       0.81      0.83      0.82        71
weighted a

## Pregunta 3.2

¿Cuál estrategia de sampling entrega mejores resultados para la clase minoritaria? 



### RESPUESTA A PREGUNTA 3.2

La mejor estrategia según el experimento hecho fue utilizando la estrategia de oversampling, que agrega datos extras a la clase minoritaria, pero obteniéndolos del mismo conjunto de training.


## Pregunta 3.3

Indique una desventaja de usar oversampling y una desventaja de usar subsampling en clasificación.


### RESPUESTA A PREGUNTA 3.3

Oversampling: A pesar de que esta estrategia entregó los mejores resultados, la desventaja es que está utilizando datos del mismo conjunto de entrenamiento para balancear la cantidad de clases. Esto provoca que el modelo pueda tener probemas de generalización, como un overfitting probando con nuevos datos.


Subsampling: La desventaja de esto es que al sacar datos para el entrenamiento puede que el modelo también tenga problemas de generalización como underfitting, debido a que en el entrenamiento se saco una cantidad significativa de datos representativos de la clase mayoritaria. 

